## **Clinical Notes Question&Answer | BART and BERT Models**

### **Clinical Notes Dataset Overview**

**UPLOAD DATASET**

The dataset consists of timestamped clinical notes recorded by healthcare providers, primarily nurses, during patient encounters and during ED visits. Each note captures specific observations, interventions, or updates made by clinical staff, particularly nurses

In [84]:
import pandas as pd

# Load the CSV file
file_path = '/content/sample_data/Patient-3-NurseNotes - Q&A.csv'
df = pd.read_csv(file_path)

In [85]:
df.head()

,VisitID,NoteSeqID,Type,ServiceDateTime,FullNoteText
0,PAT3,1,Emergency Department Notes,3/3/2025 14:38,Pt BP elevated; RN notified MD Tummala; orders...
1,PAT3,2,Emergency Department Notes,3/3/2025 14:53,labatolol held bp 139/99 mmHg
2,PAT3,3,Emergency Department Notes,3/3/2025 15:38,Pt keeps getting out of bed multiple times. Vi...
3,PAT3,4,Emergency Department Notes,3/3/2025 16:10,Pt still trying to get out of bed. Ativan; no ...
4,PAT3,5,Emergency Department Notes,3/3/2025 17:05,NP Opkan at bedside assessing the pt.


**Notes Preprocessing**

In [86]:
import nltk
import re
import string
from nltk.tokenize import word_tokenize

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('punkt_tab')

def preprocess_text(text, to_lowercase=True):
    if to_lowercase:
        text = text.lower()

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)

    # Remove mentions and hashtags
    text = re.sub(r'@\w+|#', '', text)

    # Remove all punctuation except comma and period
    punct_to_remove = string.punctuation.replace(',', '').replace('.', '')
    text = text.translate(str.maketrans('', '', punct_to_remove))

    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    # Tokenize
    tokens = word_tokenize(text)

    return ' '.join(tokens)

# Apply to DataFrame column
df['NurseNotes_cleanText'] = df['FullNoteText'].apply(preprocess_text)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [87]:
# Print the first few rows of the "NurseNotes_cleanText" column
df.head()

,VisitID,NoteSeqID,Type,ServiceDateTime,FullNoteText,NurseNotes_cleanText
0,PAT3,1,Emergency Department Notes,3/3/2025 14:38,Pt BP elevated; RN notified MD Tummala; orders...,pt bp elevated rn notified md tummala orders i...
1,PAT3,2,Emergency Department Notes,3/3/2025 14:53,labatolol held bp 139/99 mmHg,labatolol held bp 13999 mmhg
2,PAT3,3,Emergency Department Notes,3/3/2025 15:38,Pt keeps getting out of bed multiple times. Vi...,pt keeps getting out of bed multiple times . v...
3,PAT3,4,Emergency Department Notes,3/3/2025 16:10,Pt still trying to get out of bed. Ativan; no ...,pt still trying to get out of bed . ativan no ...
4,PAT3,5,Emergency Department Notes,3/3/2025 17:05,NP Opkan at bedside assessing the pt.,np opkan at bedside assessing the pt .


**Remove repeated phrases to reduce redundancy in clinical notes**

In [88]:
# Define the function to remove repeated phrases
def remove_repeated_phrases(text):
    if pd.isna(text):
        return text
    pattern = r'\b(\w+(?:\s+\w+){0,3})\b(?:\s+\1\b)+'
    return re.sub(pattern, r'\1', text, flags=re.IGNORECASE)

# Apply to the column
df['NurseNotes_phraseText'] = df['NurseNotes_cleanText'].apply(remove_repeated_phrases)

# Print the first few rows of the "NurseNotes_phraseText" column - removed repeated phrases in the text.
df.head()

,VisitID,NoteSeqID,Type,ServiceDateTime,FullNoteText,NurseNotes_cleanText,NurseNotes_phraseText
0,PAT3,1,Emergency Department Notes,3/3/2025 14:38,Pt BP elevated; RN notified MD Tummala; orders...,pt bp elevated rn notified md tummala orders i...,pt bp elevated rn notified md tummala orders i...
1,PAT3,2,Emergency Department Notes,3/3/2025 14:53,labatolol held bp 139/99 mmHg,labatolol held bp 13999 mmhg,labatolol held bp 13999 mmhg
2,PAT3,3,Emergency Department Notes,3/3/2025 15:38,Pt keeps getting out of bed multiple times. Vi...,pt keeps getting out of bed multiple times . v...,pt keeps getting out of bed multiple times . v...
3,PAT3,4,Emergency Department Notes,3/3/2025 16:10,Pt still trying to get out of bed. Ativan; no ...,pt still trying to get out of bed . ativan no ...,pt still trying to get out of bed . ativan no ...
4,PAT3,5,Emergency Department Notes,3/3/2025 17:05,NP Opkan at bedside assessing the pt.,np opkan at bedside assessing the pt .,np opkan at bedside assessing the pt .


**pyspellchecker to detect and correct spelling errors in clinical notes**

In [89]:
#!pip install pyspellchecker
#!pip install pyspellchecker==0.8.2

In [90]:
from spellchecker import SpellChecker

spell = SpellChecker()

def spell_check(text):
    words = text.split()
    corrected_words = []
    for word in words:
        # Only correct if it's alphabetic AND longer than 10 characters
        if word.isalpha() and len(word) >= 8:
            corrected = spell.correction(word)
            corrected_words.append(corrected if corrected else word)
        else:
            corrected_words.append(word)
    return ' '.join(corrected_words)

# Apply to DataFrame
df['NurseNotes_spellchecked'] = df['NurseNotes_phraseText'].apply(spell_check)

In [91]:
# Final Clean Notes - df overview
df.head()

,VisitID,NoteSeqID,Type,ServiceDateTime,FullNoteText,NurseNotes_cleanText,NurseNotes_phraseText,NurseNotes_spellchecked
0,PAT3,1,Emergency Department Notes,3/3/2025 14:38,Pt BP elevated; RN notified MD Tummala; orders...,pt bp elevated rn notified md tummala orders i...,pt bp elevated rn notified md tummala orders i...,pt bp elevated rn notified md tummala orders i...
1,PAT3,2,Emergency Department Notes,3/3/2025 14:53,labatolol held bp 139/99 mmHg,labatolol held bp 13999 mmhg,labatolol held bp 13999 mmhg,labatolol held bp 13999 mmhg
2,PAT3,3,Emergency Department Notes,3/3/2025 15:38,Pt keeps getting out of bed multiple times. Vi...,pt keeps getting out of bed multiple times . v...,pt keeps getting out of bed multiple times . v...,pt keeps getting out of bed multiple times . v...
3,PAT3,4,Emergency Department Notes,3/3/2025 16:10,Pt still trying to get out of bed. Ativan; no ...,pt still trying to get out of bed . ativan no ...,pt still trying to get out of bed . ativan no ...,pt still trying to get out of bed . ativan no ...
4,PAT3,5,Emergency Department Notes,3/3/2025 17:05,NP Opkan at bedside assessing the pt.,np opkan at bedside assessing the pt .,np opkan at bedside assessing the pt .,np opkan at bedside assessing the pt .


**--------------------------------------------------------------------------------------------------------------------------------------------------------------**

### **Group Notes Day-Wise**

In [92]:
# Combine the ServiceDateTime and NurseNotes_spellchecked into a new column called FullNotes
df['FullNotes'] = df['ServiceDateTime'].apply(lambda x: f"On {x.split()[0]} at {x.split()[1]}") + " " + df['NurseNotes_spellchecked']

# Print the first few rows of the "FullNotes" column
df.head()

,VisitID,NoteSeqID,Type,ServiceDateTime,FullNoteText,NurseNotes_cleanText,NurseNotes_phraseText,NurseNotes_spellchecked,FullNotes
0,PAT3,1,Emergency Department Notes,3/3/2025 14:38,Pt BP elevated; RN notified MD Tummala; orders...,pt bp elevated rn notified md tummala orders i...,pt bp elevated rn notified md tummala orders i...,pt bp elevated rn notified md tummala orders i...,On 3/3/2025 at 14:38 pt bp elevated rn notifie...
1,PAT3,2,Emergency Department Notes,3/3/2025 14:53,labatolol held bp 139/99 mmHg,labatolol held bp 13999 mmhg,labatolol held bp 13999 mmhg,labatolol held bp 13999 mmhg,On 3/3/2025 at 14:53 labatolol held bp 13999 mmhg
2,PAT3,3,Emergency Department Notes,3/3/2025 15:38,Pt keeps getting out of bed multiple times. Vi...,pt keeps getting out of bed multiple times . v...,pt keeps getting out of bed multiple times . v...,pt keeps getting out of bed multiple times . v...,On 3/3/2025 at 15:38 pt keeps getting out of b...
3,PAT3,4,Emergency Department Notes,3/3/2025 16:10,Pt still trying to get out of bed. Ativan; no ...,pt still trying to get out of bed . ativan no ...,pt still trying to get out of bed . ativan no ...,pt still trying to get out of bed . ativan no ...,On 3/3/2025 at 16:10 pt still trying to get ou...
4,PAT3,5,Emergency Department Notes,3/3/2025 17:05,NP Opkan at bedside assessing the pt.,np opkan at bedside assessing the pt .,np opkan at bedside assessing the pt .,np opkan at bedside assessing the pt .,On 3/3/2025 at 17:05 np opkan at bedside asses...


In [93]:
import pandas as pd

# Convert datetime column and extract date
df['ServiceDateTime'] = pd.to_datetime(df['ServiceDateTime'], errors='coerce')
df['ServiceDate'] = df['ServiceDateTime'].dt.date

# Clean notes: convert to string, strip, drop empty values
df['FullNotes'] = df['FullNotes'].astype(str).str.strip()
df = df[df['FullNotes'].str.len() > 0]  # Drop empty notes

# Group by ServiceDate and merge notes
merged_notes_df = df.groupby('ServiceDate')['FullNotes'].apply(lambda x: ' '.join(x)).reset_index()
merged_notes_df.columns = ['ServiceDate', 'MergedNote']

# Preview
merged_notes_df.head()


,ServiceDate,MergedNote
0,2025-03-03,On 3/3/2025 at 14:38 pt bp elevated rn notifie...
1,2025-03-04,On 3/4/2025 at 3:40 patient is sleeping . no s...
2,2025-03-05,On 3/5/2025 at 7:30 patient sleeping eyes clos...
3,2025-03-06,On 3/6/2025 at 7:30 opening shift note assumed...
4,2025-03-07,On 3/7/2025 at 7:09 closing shift note endorse...


In [94]:
import re

def clean_merged_note(text):
    # Step 1: Remove duplicate "On MM/DD/YYYY" patterns
    seen_dates = set()
    date_results = []

    for match in re.finditer(r'On \d{1,2}/\d{1,2}/\d{4}', text):
        date_str = match.group()
        if date_str not in seen_dates:
            seen_dates.add(date_str)
            date_results.append((match.start(), match.end(), date_str))
        else:
            date_results.append((match.start(), match.end(), ''))  # Mark for deletion

    for start, end, replacement in reversed(date_results):
        text = text[:start] + replacement + text[end:]

    # Step 2: Remove exact duplicate phrases or sentences
    # Split by sentence or phrase delimiters like period, semicolon, newline
    split_phrases = re.split(r'(?<=[.;])\s+|\n+', text.strip())

    seen_phrases = set()
    cleaned_phrases = []

    for phrase in split_phrases:
        cleaned = phrase.strip()
        if cleaned and cleaned.lower() not in seen_phrases:
            seen_phrases.add(cleaned.lower())
            cleaned_phrases.append(cleaned)

    return ' '.join(cleaned_phrases)


merged_notes_df['DaywiseNotes'] = merged_notes_df['MergedNote'].apply(clean_merged_note)

merged_notes_df[['ServiceDate', 'DaywiseNotes']].head()

,ServiceDate,DaywiseNotes
0,2025-03-03,On 3/3/2025 at 14:38 pt bp elevated rn notifie...
1,2025-03-04,On 3/4/2025 at 3:40 patient is sleeping . no s...
2,2025-03-05,On 3/5/2025 at 7:30 patient sleeping eyes clos...
3,2025-03-06,On 3/6/2025 at 7:30 opening shift note assumed...
4,2025-03-07,On 3/7/2025 at 7:09 closing shift note endorse...


In [95]:
from datetime import datetime

# Generate today's date string
today_str = datetime.today().strftime('%m%d%Y')

# Define filename with today's date
filename = f'Final-QA-Notes-{today_str}.csv'

# Save DataFrame to CSV
merged_notes_df.to_csv(filename, index=False)

# If using Google Colab, download the file
from google.colab import files
files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## **T5 - Question & Answer Model**

In [96]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datetime import date

In [97]:
# Load the FLAN-T5 model (or any T5 QA model)
model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [98]:
# Finalized restraint-related questions
questions = [
    "Answer only if 'initiated' is mentioned. Otherwise, reply 'No mention of initiated. When were the restraints initiated?",
    "What type of restraints were applied?",
    "Why were the restraints applied?",
    "Was a physician or provider notified or present at the time of restraint?",
    "Who was the provider or nurse involved in the restraint assessment or order?",
    "Was the bed alarm used for patient safety?",
    "What comfort measures were attempted before or during restraint?",
    "Answer only if 'renew' or 'renewed' or 'renewal' is mentioned. Otherwise, reply 'No mention of renewal. When was the restraint order renewed (if applicable)?",
    "Answer only if 'discontinue' or 'discontinued' is mentioned. Otherwise, reply 'No mention of discontinued. When were the restraints discontinued?",
    "What was the patient’s behavior at the time of restraint discontinuation?"
]

### **Sample Notes - 1 : Day-1**

In [99]:
sample_note = """
Date: 3/3/2025
14:38 - Patient's BP elevated. RN notified MD Tummala; orders initiated.
14:53 - Labetalol held. BP 139/99 mmHg.
15:38 - Patient keeps getting out of bed. Restless and agitated. RN notified MD Tummala again; medication ordered for comfort.
16:10 - Patient still trying to get out of bed. Ativan administered; no effect. Plan to initiate restraints if non-cooperative.
17:05 - NP Okpan assessed the patient.
17:20 - Soft wrist and ankle restraints initiated due to combative behavior and non-compliance. Patient pulling out lines, repeatedly getting out of bed. Neuro: A&OX4. PERRLA normal. CNL noted.
17:47 - Restraints reiterated. Patient now A&OX2 (oriented to self and place).
17:48 - MD Tummala at bedside. Patient yelling. Additional orders initiated.
18:33 - Patient remains restless in soft restraints. Pulses palpable. VSS. Patient confused.
19:30 - Patient admitted from ED with fall injury (facial contusion, edema). No respiratory distress. PMH: TBI, HTN, DM, schizophrenia, seizures. IV placed. Monitoring initiated.
19:30 - Patient continuesa to be restless in soft restraints.
21:40 - Patient calm and sleepy. Initially placed in 4-point restraints due to ER behavior (agitation, combative, pulling lines, non-compliance). 4-point restraints discontinued. Soft restraints applied to both wrists. Medical non-violent restraint order obtained from NP Okpan. Continued restraint monitoring planned. Restraint assessment completed. Family not reachable.

"""

# QA function
def answer_question(note, question, max_length=64):
    input_text = f"question: {question} context: {note}"
    inputs = tokenizer.encode(input_text, return_tensors="pt", truncation=True, max_length=512)

    with torch.no_grad():
        output_ids = model.generate(
            inputs,
            max_length=max_length,
            num_beams=8,
            early_stopping=True
        )
    answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return answer

# Run QA on the single sample note
qa_results = {'ServiceDate': "03/03/2025", 'DaywiseNotes': sample_note}
for i, question in enumerate(questions):
    qa_results[f"Q{i+1}"] = answer_question(sample_note, question)

# Display the results in a DataFrame
df_1 = pd.DataFrame([qa_results])

# Display answers directly in Colab
df_1


,ServiceDate,DaywiseNotes,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10
0,03/03/2025,\nDate: 3/3/2025\n14:38 - Patient's BP elevate...,17:20,Soft wrist and ankle,combative behavior and non-compliance,MD Tummala,NP Okpan,yes,Ativan administered,No mention of renewal,21:40,calm and sleepy


### **Sample Notes - 2 : Day-2**

In [100]:
sample_note = """
Date: 3/4/2025
03:40 - Patient is sleeping. No signs of distress or SOB. Attempted to wake but very sleepy. Sitter bedside. Bed alarm on for safety. Monitoring continues.
07:40 - Soft restraints discontinued. Patient no longer pulling lines/tubes. Still confused but reorientable. Skin intact. Patient A&Ox2. Offered nutrition, hydration, repositioning, hygiene. Safety precautions in place: bed in lowest position, 2 side rails, call light reachable, bed alarm on, sitter present.
10:20 - Patient increasingly agitated, pulling lines, trying to get out of bed, hitting and kicking staff. Reorientation and distractions failed. Ativan administered. MD notified. Sitter at bedside.
10:30 - Agitation continues. Patient yelling and attempting to strike staff. Dr. Testa notified and new orders received.
11:00 - Dr. Testa at bedside.
11:10 - Patient placed in 4-point soft restraints due to extreme agitation and combative behavior. All comfort measures failed. Dr. Testa notified. Order for restraints obtained.
        Patient educated on use and safety. Restraints applied.
11:48 - Dr. Testa on unit. New order placed.
13:10 - Restraints continued. Agitation persists. All comfort measures failed. Skin intact. Patient A&Ox1, unable to comprehend instructions due to ALOC/agitation. Safety precautions maintained. Nutrition, repositioning, and hygiene offered. Sitter present.
15:10 - Restraint renewal order placed. Patient continues pulling at lines and exhibiting aggressive behavior. All comfort measures ineffective. A&Ox1. Patient unable to comprehend instructions. Skin intact. All safety protocols followed.
17:10 - Restraints continued. No improvement in agitation. Measures repeated. Skin intact. A&Ox1. Repositioned and hydrated. Sitter present.
19:10 - Another restraint renewal order. Same behavioral pattern. Skin intact. A&Ox1. Patient unable to understand safety instructions. Continued monitoring and care.
21:10 - Restraints continued. Behavior unchanged. No redness or skin breakdown. Patient remains A&Ox. Safety precautions enforced.
23:00 - Soft restraints discontinued. Patient AOx2, still mildly confused but no longer combative or pulling IVs. Skin assessed — no issues. Patient reoriented and repositioned. Hydration/nutrition offered. Patient resting. Bed in lowest position, brakes on, side rails up, call light reachable, sitter at bedside.
"""

# QA function
def answer_question(note, question, max_length=64):
    input_text = f"question: {question} context: {note}"
    inputs = tokenizer.encode(input_text, return_tensors="pt", truncation=True, max_length=512)

    with torch.no_grad():
        output_ids = model.generate(
            inputs,
            max_length=max_length,
            num_beams=8,
            early_stopping=True
        )
    answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return answer

# Run QA on the single sample note
qa_results = {'ServiceDate': "03/04/2025", 'DaywiseNotes': sample_note}
for i, question in enumerate(questions):
    qa_results[f"Q{i+1}"] = answer_question(sample_note, question)

# Display the results in a DataFrame
df_2 = pd.DataFrame([qa_results])

# Display answers directly in Colab
df_2


,ServiceDate,DaywiseNotes,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10
0,03/04/2025,\nDate: 3/4/2025\n03:40 - Patient is sleeping....,11:10,4-point soft,Patient placed in 4-point soft restraints due ...,Dr. Testa,Dr. Testa,Yes,"Nutrition, repositioning, and hygiene offered....",15:10,07:40,pulling lines/tubes


### **Sample Notes - 3 : Day-3**

In [101]:
sample_note = """
Date: 3/5/2025
07:30 - Patient sleeping with eyes closed. No signs of distress. Sitter at bedside.
09:44 - Breakfast fed and tolerated well. Medications given as ordered. Linen changed. Patient remains confused, unable to state name, date of birth, or location. Continues attempting to get out of bed. Sitter at bedside.
10:26 - Patient yelling, confused, and trying to get out of bed. Sitter at bedside.
18:25 - Patient agitated, yelling, and attempting to get out of bed. Medicated with Ativan. Plan of care continued.
20:34 - Patient agitated and trying to get out of bed. Hydration and repositioning offered. Patient was redirected and educated on use of telemetry box and gown. Patient cooperated and rested after intervention. Sitter at bedside. Plan of care continued.
"""
# QA function
def answer_question(note, question, max_length=64):
    input_text = f"question: {question} context: {note}"
    inputs = tokenizer.encode(input_text, return_tensors="pt", truncation=True, max_length=512)

    with torch.no_grad():
        output_ids = model.generate(
            inputs,
            max_length=max_length,
            num_beams=8,
            early_stopping=True
        )
    answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return answer

# Run QA on the single sample note
qa_results = {'ServiceDate': "03/05/2025", 'DaywiseNotes': sample_note}
for i, question in enumerate(questions):
    qa_results[f"Q{i+1}"] = answer_question(sample_note, question)

# Display the results in a DataFrame
df_3 = pd.DataFrame([qa_results])

# Display answers directly in Colab
df_3



,ServiceDate,DaywiseNotes,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10
0,03/05/2025,\nDate: 3/5/2025\n07:30 - Patient sleeping wit...,No mention of initiated,telemetry box,Patient agitated and trying to get out of bed,notified,Sitter,yes,sitter at bedside,No mention of renewal,No mention of discontinued,Patient agitated and trying to get out of bed


### **Sample Notes - 4 : Day-4**

In [102]:
sample_note = """
Date: 3/6/2025
07:30 - Opening shift note. Assumed care of patient. Patient awake and alert. No signs of distress or shortness of breath. Patient out of bed, yelling at CNA, attempting to leave room. Charge nurse David notified. Bed in lowest locked position, side rails up x2, call light within reach. Care ongoing.
08:05 - Report given to Edididong RN. Patient at bedside eating breakfast with no signs of distress or pain. Care ongoing.
19:58 - Patient agitated, pulling on leads, trying to jump off bed, hitting aide.
"""

# QA function
def answer_question(note, question, max_length=64):
    input_text = f"question: {question} context: {note}"
    inputs = tokenizer.encode(input_text, return_tensors="pt", truncation=True, max_length=512)

    with torch.no_grad():
        output_ids = model.generate(
            inputs,
            max_length=max_length,
            num_beams=8,
            early_stopping=True
        )
    answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return answer

# Run QA on the single sample note
qa_results = {'ServiceDate': "03/06/2025", 'DaywiseNotes': sample_note}
for i, question in enumerate(questions):
    qa_results[f"Q{i+1}"] = answer_question(sample_note, question)

# Display the results in a DataFrame
df_4 = pd.DataFrame([qa_results])

# Display answers directly in Colab
df_4

,ServiceDate,DaywiseNotes,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10
0,03/06/2025,\nDate: 3/6/2025\n07:30 - Opening shift note. ...,19:58,leads,attempting to leave room,notified,Edididong,yes,No signs of distress or shortness of breath,No mention of renewal,No mention of discontinued,agitated


### **Sample Notes - 5 : Day-5**

In [103]:
sample_note = """
Date: 3/7/2025
07:09 - Closing shift note. Care endorsed to day shift.
19:20 - Opening shift note. Assumed care of patient. Patient is awake, confused, and uncooperative. No signs of distress, shortness of breath, or pain. All safety precautions in place. Call light within reach. Sitter at bedside.
20:48 - Handoff report given to night shift nurse Elias for continued care.
"""

# QA function
def answer_question(note, question, max_length=64):
    input_text = f"question: {question} context: {note}"
    inputs = tokenizer.encode(input_text, return_tensors="pt", truncation=True, max_length=512)

    with torch.no_grad():
        output_ids = model.generate(
            inputs,
            max_length=max_length,
            num_beams=5,
            early_stopping=True
        )
    answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return answer

# Run QA on the single sample note
qa_results = {'ServiceDate': "03/07/2025", 'DaywiseNotes': sample_note}
for i, question in enumerate(questions):
    qa_results[f"Q{i+1}"] = answer_question(sample_note, question)

# Display the results in a DataFrame
df_5 = pd.DataFrame([qa_results])

# Display answers directly in Colab
df_5


,ServiceDate,DaywiseNotes,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10
0,03/07/2025,\nDate: 3/7/2025\n07:09 - Closing shift note. ...,No mention of initiated,All safety precautions in place,"awake, confused, and uncooperative",no,Elias,Yes,Sitter at bedside,No mention of renewal,No mention of discontinued,uncooperative


### **Sample Notes - 6 : Day-6**

In [104]:
sample_note = """
Date: 3/8/2025
06:26 - Blood glucose 53 (first drop into the 40s). Patient sleeping but easily aroused, not cooperating with drinking juice. D50 IV administered as ordered. MD paged. Patient under close monitoring.
18:10 - Report called to Joyce, RN. Patient to be transferred to Room 220A.
18:19 - Patient transferred to unit in stable condition. Sitter at bedside.
"""

# QA function
def answer_question(note, question, max_length=64):
    input_text = f"question: {question} context: {note}"
    inputs = tokenizer.encode(input_text, return_tensors="pt", truncation=True, max_length=512)

    with torch.no_grad():
        output_ids = model.generate(
            inputs,
            max_length=max_length,
            num_beams=8,
            early_stopping=True
        )
    answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return answer

# Run QA on the single sample note
qa_results = {'ServiceDate': "03/08/2025", 'DaywiseNotes': sample_note}
for i, question in enumerate(questions):
    qa_results[f"Q{i+1}"] = answer_question(sample_note, question)

# Display the results in a DataFrame
df_6 = pd.DataFrame([qa_results])

# Display answers directly in Colab
df_6

,ServiceDate,DaywiseNotes,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10
0,03/08/2025,\nDate: 3/8/2025\n06:26 - Blood glucose 53 (fi...,No mention of initiated,Sitter,not cooperating with drinking juice,MD paged,MD,yes,sitter at bedside,No mention of renewal,No mention of discontinued,sleeping but easily aroused


### **Sample Notes - 7 : Day-7**

In [105]:
sample_note = """
Date: 3/10/2025
13:58 – Resident MD Moreno rounding. Informed that morning medications were held as patient was asleep most of the day and became verbally incomprehensible when awakened. Patient yelled "I sleep" and resisted touch. Order placed for tele psych consult.
14:07 – Tele psych completed. MD Oliver recommended increasing Seroquel dosage to 300 mg and advised communication with resident MD. MD Cruz Moreno was informed.
"""

# QA function
def answer_question(note, question, max_length=64):
    input_text = f"question: {question} context: {note}"
    inputs = tokenizer.encode(input_text, return_tensors="pt", truncation=True, max_length=512)

    with torch.no_grad():
        output_ids = model.generate(
            inputs,
            max_length=max_length,
            num_beams=8,
            early_stopping=True
        )
    answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return answer

# Run QA on the single sample note
qa_results = {'ServiceDate': "03/10/2025", 'DaywiseNotes': sample_note}
for i, question in enumerate(questions):
    qa_results[f"Q{i+1}"] = answer_question(sample_note, question)

# Display the results in a DataFrame
df_7 = pd.DataFrame([qa_results])

# Display answers directly in Colab
df_7

,ServiceDate,DaywiseNotes,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10
0,03/10/2025,\nDate: 3/10/2025\n13:58 – Resident MD Moreno ...,No mention of initiated.,touch,patient was asleep most of the day,Informed,Resident MD Moreno,yes,touch,No mention of renewal,No mention of discontinued,verbally incomprehensible


### **Sample Notes - 8 : Day-8**

In [106]:
sample_note = """
Date: 3/11/2025
10:36 – Resident MD Cruz Moreno rounding. Patient is more awake today, took morning medications, noncombative, and less yelling compared to previous days. New orders received.
21:30 – New orders received from Dr. Zhang due to patient agitation and combativeness.
"""

# QA function
def answer_question(note, question, max_length=64):
    input_text = f"question: {question} context: {note}"
    inputs = tokenizer.encode(input_text, return_tensors="pt", truncation=True, max_length=512)

    with torch.no_grad():
        output_ids = model.generate(
            inputs,
            max_length=max_length,
            num_beams=8,
            early_stopping=True
        )
    answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return answer

# Run QA on the single sample note
qa_results = {'ServiceDate': "03/11/2025", 'DaywiseNotes': sample_note}
for i, question in enumerate(questions):
    qa_results[f"Q{i+1}"] = answer_question(sample_note, question)

# Display the results in a DataFrame
df_8 = pd.DataFrame([qa_results])

# Display answers directly in Colab
df_8

,ServiceDate,DaywiseNotes,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10
0,03/11/2025,\nDate: 3/11/2025\n10:36 – Resident MD Cruz Mo...,No mention of initiated,noncombative,patient agitation and combativeness,notified,Dr Zhang,yes,noncombative,No mention of renewal,No mention of,agitation and combativeness


### **Sample Notes - 9 : Day-9**

In [107]:
sample_note = """
Date: 3/12/2025
08:00 – Opening shift note. Assumed care of patient. Patient is awake and alert. Bed is in lowest locked position with side rails up x2 and call light within reach. No signs of distress, SOB, or pain. Patient instructed on plan of care and to call for assistance as needed. Sitter is at bedside.
10:00 – Per Dr. Cruz, Lisinopril was held.
17:31 – Tele box returned to monitor tech.
"""

# QA function
def answer_question(note, question, max_length=64):
    input_text = f"question: {question} context: {note}"
    inputs = tokenizer.encode(input_text, return_tensors="pt", truncation=True, max_length=512)

    with torch.no_grad():
        output_ids = model.generate(
            inputs,
            max_length=max_length,
            num_beams=8,
            early_stopping=True
        )
    answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return answer

# Run QA on the single sample note
qa_results = {'ServiceDate': "03/12/2025", 'DaywiseNotes': sample_note}
for i, question in enumerate(questions):
    qa_results[f"Q{i+1}"] = answer_question(sample_note, question)

# Display the results in a DataFrame
df_9 = pd.DataFrame([qa_results])

# Display answers directly in Colab
df_9

,ServiceDate,DaywiseNotes,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10
0,03/12/2025,\nDate: 3/12/2025\n08:00 – Opening shift note....,No mention of initiated,side rails,patient is awake and alert,Dr. Cruz,Dr. Cruz,yes,Sitter is at bedside,No mention of renewal,No mention of discontinued,awake and alert


### **Sample Notes - 10 : Day-10**

In [108]:
sample_note = """
Date: 3/13/2025
09:58 – Resident rounds by MD Cruz. Patient mittens removed, patient pleasant. No new orders.
10:15 – In process of obtaining SCDs.
13:50 – Attempted to give 1400 medications. Family at bedside requested delay as patient had just fallen asleep. RN acknowledged.
14:34 – Follow-up visit; patient still asleep. New family member requested RN to return later.
16:10 – IV removal: 22G in right forearm and 20G in left forearm removed with sterile technique. Catheters intact, pressure dressing applied. Patient tolerated procedure well.
16:14 – Spoke with Dr. Siddiqui via phone; approved change in discharge disposition to “Home.”
16:28 – Transportation arrived at bedside.
16:41 – Discharge completed. Instructions given and all questions addressed. Medication reconciliation form provided. Patient escorted to vehicle via gurney with all belongings, no signs of pain or distress. Accompanied by GreenMed transport.
"""

# QA function
def answer_question(note, question, max_length=64):
    input_text = f"question: {question} context: {note}"
    inputs = tokenizer.encode(input_text, return_tensors="pt", truncation=True, max_length=512)

    with torch.no_grad():
        output_ids = model.generate(
            inputs,
            max_length=max_length,
            num_beams=8,
            early_stopping=True
        )
    answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return answer

# Run QA on the single sample note
qa_results = {'ServiceDate': "03/13/2025", 'DaywiseNotes': sample_note}
for i, question in enumerate(questions):
    qa_results[f"Q{i+1}"] = answer_question(sample_note, question)

# Display the results in a DataFrame
df_10 = pd.DataFrame([qa_results])

# Display answers directly in Colab
df_10

,ServiceDate,DaywiseNotes,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10
0,03/13/2025,\nDate: 3/13/2025\n09:58 – Resident rounds by ...,No mention of initiated.,gurney,patient had just fallen asleep,RN acknowledged,MD Cruz,yes,pressure dressing applied,No mention of renewal,No mention of discontinued.,patient pleasant


## **--------------------T5 Final Q&A Results--------------------**

In [109]:
# Step 1: Create a list of all dataframes
all_dfs = [df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8, df_9, df_10]

# Step 2: Concatenate into a single DataFrame
df_combined_T5 = pd.concat(all_dfs, ignore_index=True)

# Step 3: Display the combined result
display(df_combined_T5)


,ServiceDate,DaywiseNotes,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10
0,03/03/2025,\nDate: 3/3/2025\n14:38 - Patient's BP elevate...,17:20,Soft wrist and ankle,combative behavior and non-compliance,MD Tummala,NP Okpan,yes,Ativan administered,No mention of renewal,21:40,calm and sleepy
1,03/04/2025,\nDate: 3/4/2025\n03:40 - Patient is sleeping....,11:10,4-point soft,Patient placed in 4-point soft restraints due ...,Dr. Testa,Dr. Testa,Yes,"Nutrition, repositioning, and hygiene offered....",15:10,07:40,pulling lines/tubes
2,03/05/2025,\nDate: 3/5/2025\n07:30 - Patient sleeping wit...,No mention of initiated,telemetry box,Patient agitated and trying to get out of bed,notified,Sitter,yes,sitter at bedside,No mention of renewal,No mention of discontinued,Patient agitated and trying to get out of bed
3,03/06/2025,\nDate: 3/6/2025\n07:30 - Opening shift note. ...,19:58,leads,attempting to leave room,notified,Edididong,yes,No signs of distress or shortness of breath,No mention of renewal,No mention of discontinued,agitated
4,03/07/2025,\nDate: 3/7/2025\n07:09 - Closing shift note. ...,No mention of initiated,All safety precautions in place,"awake, confused, and uncooperative",no,Elias,Yes,Sitter at bedside,No mention of renewal,No mention of discontinued,uncooperative
5,03/08/2025,\nDate: 3/8/2025\n06:26 - Blood glucose 53 (fi...,No mention of initiated,Sitter,not cooperating with drinking juice,MD paged,MD,yes,sitter at bedside,No mention of renewal,No mention of discontinued,sleeping but easily aroused
6,03/10/2025,\nDate: 3/10/2025\n13:58 – Resident MD Moreno ...,No mention of initiated.,touch,patient was asleep most of the day,Informed,Resident MD Moreno,yes,touch,No mention of renewal,No mention of discontinued,verbally incomprehensible
7,03/11/2025,\nDate: 3/11/2025\n10:36 – Resident MD Cruz Mo...,No mention of initiated,noncombative,patient agitation and combativeness,notified,Dr Zhang,yes,noncombative,No mention of renewal,No mention of,agitation and combativeness
8,03/12/2025,\nDate: 3/12/2025\n08:00 – Opening shift note....,No mention of initiated,side rails,patient is awake and alert,Dr. Cruz,Dr. Cruz,yes,Sitter is at bedside,No mention of renewal,No mention of discontinued,awake and alert
9,03/13/2025,\nDate: 3/13/2025\n09:58 – Resident rounds by ...,No mention of initiated.,gurney,patient had just fallen asleep,RN acknowledged,MD Cruz,yes,pressure dressing applied,No mention of renewal,No mention of discontinued.,patient pleasant


In [110]:
from datetime import datetime

# Generate today's date string
today_str = datetime.today().strftime('%m%d%Y')

# Define filename with today's date
filename = f'T5_QA-Answers-{today_str}.csv'

# Save DataFrame to CSV
df_combined_T5.to_csv(filename, index=False)

# If using Google Colab, download the file
from google.colab import files
files.download(filename)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## **DistilBERT - Question & Answer Model**

In [111]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

In [112]:
# Load the FLAN-T5 model (or any T5 QA model)
model_name = "distilbert-base-uncased-distilled-squad"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

In [113]:
# Finalized restraint-related questions
questions = [
    "Answer only if 'initiated' is mentioned. Otherwise, reply 'No mention of initiated. When were the restraints initiated?",
    "What type of restraints were applied?",
    "Why were the restraints applied?",
    "Was a physician or provider notified or present at the time of restraint?",
    "Who was the provider or nurse involved in the restraint assessment or order?",
    "Was the bed alarm used for patient safety?",
    "What comfort measures were attempted before or during restraint?",
    "Answer only if 'renew' or 'renewed' or 'renewal' is mentioned. Otherwise, reply 'No mention of renewal. When was the restraint order renewed (if applicable)?",
    "Answer only if 'discontinue' or 'discontinued' is mentioned. Otherwise, reply 'No mention of discontinued. When were the restraints discontinued?",
    "What was the patient’s behavior at the time of restraint discontinuation?"
]

### **Sample Notes - 1 : Day-1**

In [114]:
sample_note = """
Date: 3/3/2025
14:38 - Patient's BP elevated. RN notified MD Tummala; orders initiated.
14:53 - Labetalol held. BP 139/99 mmHg.
15:38 - Patient keeps getting out of bed. Restless and agitated. RN notified MD Tummala again; medication ordered for comfort.
16:10 - Patient still trying to get out of bed. Ativan administered; no effect. Plan to initiate restraints if non-cooperative.
17:05 - NP Okpan assessed the patient.
17:20 - Soft wrist and ankle restraints initiated due to combative behavior and non-compliance. Patient pulling out lines, repeatedly getting out of bed. Neuro: A&OX4. PERRLA normal. CNL noted.
17:47 - Restraints reiterated. Patient now A&OX2 (oriented to self and place).
17:48 - MD Tummala at bedside. Patient yelling. Additional orders initiated.
18:33 - Patient remains restless in soft restraints. Pulses palpable. VSS. Patient confused.
19:30 - Patient admitted from ED with fall injury (facial contusion, edema). No respiratory distress. PMH: TBI, HTN, DM, schizophrenia, seizures. IV placed. Monitoring initiated.
19:30 - Patient continuesa to be restless in soft restraints.
21:40 - Patient calm and sleepy. Initially placed in 4-point restraints due to ER behavior (agitation, combative, pulling lines, non-compliance). 4-point restraints discontinued. Soft restraints applied to both wrists. Medical non-violent restraint order obtained from NP Okpan. Continued restraint monitoring planned. Restraint assessment completed. Family not reachable.
"""

# QA function (Extractive)
def answer_question(note, question):
    inputs = tokenizer.encode_plus(question, note, return_tensors="pt", truncation=True)

    with torch.no_grad():
        outputs = model(**inputs)
        start_idx = torch.argmax(outputs.start_logits)
        end_idx = torch.argmax(outputs.end_logits) + 1

    answer_ids = inputs["input_ids"][0][start_idx:end_idx]
    answer = tokenizer.decode(answer_ids, skip_special_tokens=True)
    return answer

# Run QA on the sample note
qa_results = {'ServiceDate': "03/03/2025", 'DaywiseNotes': sample_note}
for i, question in enumerate(questions):
    qa_results[f"Q{i+1}"] = answer_question(sample_note, question)

# Create DataFrame
df_1 = pd.DataFrame([qa_results])
df_1

,ServiceDate,DaywiseNotes,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10
0,03/03/2025,\nDate: 3/3/2025\n14:38 - Patient's BP elevate...,,soft restraints,,,np okpan,,,no mention of renewal,answer only if ' discontinue ' or ' discontinu...,


### **Sample Notes - 2 : Day-2**

In [115]:
sample_note = """
Date: 3/4/2025
03:40 - Patient is sleeping. No signs of distress or SOB. Attempted to wake but very sleepy. Sitter bedside. Bed alarm on for safety. Monitoring continues.
07:40 - Soft restraints discontinued. Patient no longer pulling lines/tubes. Still confused but reorientable. Skin intact. Patient A&Ox2. Offered nutrition, hydration, repositioning, hygiene. Safety precautions in place: bed in lowest position, 2 side rails, call light reachable, bed alarm on, sitter present.
10:20 - Patient increasingly agitated, pulling lines, trying to get out of bed, hitting and kicking staff. Reorientation and distractions failed. Ativan administered. MD notified. Sitter at bedside.
10:30 - Agitation continues. Patient yelling and attempting to strike staff. Dr. Testa notified and new orders received.
11:00 - Dr. Testa at bedside.
11:10 - Patient placed in 4-point soft restraints due to extreme agitation and combative behavior. All comfort measures failed. Dr. Testa notified. Order for restraints obtained.
        Patient educated on use and safety. Restraints applied.
11:48 - Dr. Testa on unit. New order placed.
13:10 - Restraints continued. Agitation persists. All comfort measures failed. Skin intact. Patient A&Ox1, unable to comprehend instructions due to ALOC/agitation. Safety precautions maintained. Nutrition, repositioning, and hygiene offered. Sitter present.
15:10 - Restraint renewal order placed. Patient continues pulling at lines and exhibiting aggressive behavior. All comfort measures ineffective. A&Ox1. Patient unable to comprehend instructions. Skin intact. All safety protocols followed.
17:10 - Restraints continued. No improvement in agitation. Measures repeated. Skin intact. A&Ox1. Repositioned and hydrated. Sitter present.
19:10 - Another restraint renewal order. Same behavioral pattern. Skin intact. A&Ox1. Patient unable to understand safety instructions. Continued monitoring and care.
21:10 - Restraints continued. Behavior unchanged. No redness or skin breakdown. Patient remains A&Ox. Safety precautions enforced.
23:00 - Soft restraints discontinued. Patient AOx2, still mildly confused but no longer combative or pulling IVs. Skin assessed — no issues. Patient reoriented and repositioned. Hydration/nutrition offered. Patient resting. Bed in lowest position, brakes on, side rails up, call light reachable, sitter at bedside.
"""

# QA function (Extractive)
def answer_question(note, question):
    inputs = tokenizer.encode_plus(question, note, return_tensors="pt", truncation=True)

    with torch.no_grad():
        outputs = model(**inputs)
        start_idx = torch.argmax(outputs.start_logits)
        end_idx = torch.argmax(outputs.end_logits) + 1

    answer_ids = inputs["input_ids"][0][start_idx:end_idx]
    answer = tokenizer.decode(answer_ids, skip_special_tokens=True)
    return answer

# Run QA on the sample note
qa_results = {'ServiceDate': "03/04/2025", 'DaywiseNotes': sample_note}
for i, question in enumerate(questions):
    qa_results[f"Q{i+1}"] = answer_question(sample_note, question)

# Create DataFrame
df_2 = pd.DataFrame([qa_results])
df_2

,ServiceDate,DaywiseNotes,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10
0,03/04/2025,\nDate: 3/4/2025\n03:40 - Patient is sleeping....,,,,,,,,,answer only if ' discontinue ' or ' discontinu...,


### **Sample Notes - 3 : Day-3**

In [116]:
sample_note = """
Date: 3/5/2025
07:30 - Patient sleeping with eyes closed. No signs of distress. Sitter at bedside.
09:44 - Breakfast fed and tolerated well. Medications given as ordered. Linen changed. Patient remains confused, unable to state name, date of birth, or location. Continues attempting to get out of bed. Sitter at bedside.
10:26 - Patient yelling, confused, and trying to get out of bed. Sitter at bedside.
18:25 - Patient agitated, yelling, and attempting to get out of bed. Medicated with Ativan. Plan of care continued.
20:34 - Patient agitated and trying to get out of bed. Hydration and repositioning offered. Patient was redirected and educated on use of telemetry box and gown. Patient cooperated and rested after intervention. Sitter at bedside. Plan of care continued.
"""

# QA function (Extractive)
def answer_question(note, question):
    inputs = tokenizer.encode_plus(question, note, return_tensors="pt", truncation=True)

    with torch.no_grad():
        outputs = model(**inputs)
        start_idx = torch.argmax(outputs.start_logits)
        end_idx = torch.argmax(outputs.end_logits) + 1

    answer_ids = inputs["input_ids"][0][start_idx:end_idx]
    answer = tokenizer.decode(answer_ids, skip_special_tokens=True)
    return answer

# Run QA on the sample note
qa_results = {'ServiceDate': "03/05/2025", 'DaywiseNotes': sample_note}
for i, question in enumerate(questions):
    qa_results[f"Q{i+1}"] = answer_question(sample_note, question)

# Create DataFrame
df_3 = pd.DataFrame([qa_results])
df_3

,ServiceDate,DaywiseNotes,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10
0,03/05/2025,\nDate: 3/5/2025\n07:30 - Patient sleeping wit...,3 / 5 / 2025,medications given as ordered,medications given as ordered,,sitter,no signs of distress,medications given as ordered. linen changed. p...,no mention of renewal,no mention of discontinued. when were the rest...,confused


### **Sample Notes - 4 : Day-4**

In [117]:
sample_note = """
Date: 3/6/2025
07:30 - Opening shift note. Assumed care of patient. Patient awake and alert. No signs of distress or shortness of breath. Patient out of bed, yelling at CNA, attempting to leave room. Charge nurse David notified. Bed in lowest locked position, side rails up x2, call light within reach. Care ongoing.
08:05 - Report given to Edididong RN. Patient at bedside eating breakfast with no signs of distress or pain. Care ongoing.
19:58 - Patient agitated, pulling on leads, trying to jump off bed, hitting aide.
"""

# QA function (Extractive)
def answer_question(note, question):
    inputs = tokenizer.encode_plus(question, note, return_tensors="pt", truncation=True)

    with torch.no_grad():
        outputs = model(**inputs)
        start_idx = torch.argmax(outputs.start_logits)
        end_idx = torch.argmax(outputs.end_logits) + 1

    answer_ids = inputs["input_ids"][0][start_idx:end_idx]
    answer = tokenizer.decode(answer_ids, skip_special_tokens=True)
    return answer

# Run QA on the sample note
qa_results = {'ServiceDate': "03/06/2025", 'DaywiseNotes': sample_note}
for i, question in enumerate(questions):
    qa_results[f"Q{i+1}"] = answer_question(sample_note, question)

# Create DataFrame
df_4 = pd.DataFrame([qa_results])
df_4

,ServiceDate,DaywiseNotes,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10
0,03/06/2025,\nDate: 3/6/2025\n07:30 - Opening shift note. ...,when were the restraints initiated? date : 3 /...,side rails up x2,no signs of distress or shortness of breath. p...,charge nurse david notified,charge nurse david notified,patient awake and alert,no signs of distress or shortness of breath,no mention of renewal,no mention of discontinued. when were the rest...,patient awake and alert


### **Sample Notes - 5 : Day-5**

In [118]:
sample_note = """
Date: 3/7/2025
07:09 - Closing shift note. Care endorsed to day shift.
19:20 - Opening shift note. Assumed care of patient. Patient is awake, confused, and uncooperative. No signs of distress, shortness of breath, or pain. All safety precautions in place. Call light within reach. Sitter at bedside.
20:48 - Handoff report given to night shift nurse Elias for continued care.
"""

# QA function (Extractive)
def answer_question(note, question):
    inputs = tokenizer.encode_plus(question, note, return_tensors="pt", truncation=True)

    with torch.no_grad():
        outputs = model(**inputs)
        start_idx = torch.argmax(outputs.start_logits)
        end_idx = torch.argmax(outputs.end_logits) + 1

    answer_ids = inputs["input_ids"][0][start_idx:end_idx]
    answer = tokenizer.decode(answer_ids, skip_special_tokens=True)
    return answer

# Run QA on the sample note
qa_results = {'ServiceDate': "03/07/2025", 'DaywiseNotes': sample_note}
for i, question in enumerate(questions):
    qa_results[f"Q{i+1}"] = answer_question(sample_note, question)

# Create DataFrame
df_5 = pd.DataFrame([qa_results])
df_5

,ServiceDate,DaywiseNotes,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10
0,03/07/2025,\nDate: 3/7/2025\n07:09 - Closing shift note. ...,when were the restraints initiated? date : 3 /...,safety precautions,all safety precautions in place,all safety precautions,night shift nurse elias,all safety precautions in place,safety precautions,no mention of renewal,no mention of discontinued,"awake, confused, and uncooperative"


### **Sample Notes - 6 : Day-6**

In [119]:
sample_note = """
Date: 3/8/2025
06:26 - Blood glucose 53 (first drop into the 40s). Patient sleeping but easily aroused, not cooperating with drinking juice. D50 IV administered as ordered. MD paged. Patient under close monitoring.
18:10 - Report called to Joyce, RN. Patient to be transferred to Room 220A.
18:19 - Patient transferred to unit in stable condition. Sitter at bedside.
"""

# QA function (Extractive)
def answer_question(note, question):
    inputs = tokenizer.encode_plus(question, note, return_tensors="pt", truncation=True)

    with torch.no_grad():
        outputs = model(**inputs)
        start_idx = torch.argmax(outputs.start_logits)
        end_idx = torch.argmax(outputs.end_logits) + 1

    answer_ids = inputs["input_ids"][0][start_idx:end_idx]
    answer = tokenizer.decode(answer_ids, skip_special_tokens=True)
    return answer

# Run QA on the sample note
qa_results = {'ServiceDate': "03/08/2025", 'DaywiseNotes': sample_note}
for i, question in enumerate(questions):
    qa_results[f"Q{i+1}"] = answer_question(sample_note, question)

# Create DataFrame
df_6 = pd.DataFrame([qa_results])
df_6

,ServiceDate,DaywiseNotes,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10
0,03/08/2025,\nDate: 3/8/2025\n06:26 - Blood glucose 53 (fi...,when were the restraints initiated? date : 3 /...,sitter,"patient sleeping but easily aroused, not coope...",close monitoring,md paged,"patient sleeping but easily aroused, not coope...",sleeping,no mention of renewal,no mention of discontinued,sleeping but easily aroused


### **Sample Notes - 7 : Day-7**

In [120]:
sample_note = """
Date: 3/10/2025
13:58 – Resident MD Moreno rounding. Informed that morning medications were held as patient was asleep most of the day and became verbally incomprehensible when awakened. Patient yelled "I sleep" and resisted touch. Order placed for tele psych consult.
14:07 – Tele psych completed. MD Oliver recommended increasing Seroquel dosage to 300 mg and advised communication with resident MD. MD Cruz Moreno was informed.
"""

# QA function (Extractive)
def answer_question(note, question):
    inputs = tokenizer.encode_plus(question, note, return_tensors="pt", truncation=True)

    with torch.no_grad():
        outputs = model(**inputs)
        start_idx = torch.argmax(outputs.start_logits)
        end_idx = torch.argmax(outputs.end_logits) + 1

    answer_ids = inputs["input_ids"][0][start_idx:end_idx]
    answer = tokenizer.decode(answer_ids, skip_special_tokens=True)
    return answer

# Run QA on the sample note
qa_results = {'ServiceDate': "03/10/2025", 'DaywiseNotes': sample_note}
for i, question in enumerate(questions):
    qa_results[f"Q{i+1}"] = answer_question(sample_note, question)

# Create DataFrame
df_7 = pd.DataFrame([qa_results])
df_7

,ServiceDate,DaywiseNotes,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10
0,03/10/2025,\nDate: 3/10/2025\n13:58 – Resident MD Moreno ...,when were the restraints initiated? date : 3 /...,medications,patient was asleep most of the day and became ...,informed that morning medications were held as...,tele psych consult,tele psych consult,medications were held as patient was asleep mo...,no mention of renewal,no mention of discontinued,


### **Sample Notes - 8 : Day-8**

In [121]:
sample_note = """
Date: 3/11/2025
10:36 – Resident MD Cruz Moreno rounding. Patient is more awake today, took morning medications, noncombative, and less yelling compared to previous days. New orders received.
21:30 – New orders received from Dr. Zhang due to patient agitation and combativeness.
"""

# QA function (Extractive)
def answer_question(note, question):
    inputs = tokenizer.encode_plus(question, note, return_tensors="pt", truncation=True)

    with torch.no_grad():
        outputs = model(**inputs)
        start_idx = torch.argmax(outputs.start_logits)
        end_idx = torch.argmax(outputs.end_logits) + 1

    answer_ids = inputs["input_ids"][0][start_idx:end_idx]
    answer = tokenizer.decode(answer_ids, skip_special_tokens=True)
    return answer

# Run QA on the sample note
qa_results = {'ServiceDate': "03/11/2025", 'DaywiseNotes': sample_note}
for i, question in enumerate(questions):
    qa_results[f"Q{i+1}"] = answer_question(sample_note, question)

# Create DataFrame
df_8 = pd.DataFrame([qa_results])
df_8

,ServiceDate,DaywiseNotes,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10
0,03/11/2025,\nDate: 3/11/2025\n10:36 – Resident MD Cruz Mo...,when were the restraints initiated? date : 3 /...,morning medications,patient agitation and combativeness,"patient is more awake today, took morning medi...",dr. zhang,"patient is more awake today, took morning medi...","morning medications, noncombative, and less ye...",no mention of renewal,no mention of discontinued,yelling


### **Sample Notes - 9 : Day-9**

In [122]:
sample_note = """
Date: 3/12/2025
08:00 – Opening shift note. Assumed care of patient. Patient is awake and alert. Bed is in lowest locked position with side rails up x2 and call light within reach. No signs of distress, SOB, or pain. Patient instructed on plan of care and to call for assistance as needed. Sitter is at bedside.
10:00 – Per Dr. Cruz, Lisinopril was held.
17:31 – Tele box returned to monitor tech.
"""

# QA function (Extractive)
def answer_question(note, question):
    inputs = tokenizer.encode_plus(question, note, return_tensors="pt", truncation=True)

    with torch.no_grad():
        outputs = model(**inputs)
        start_idx = torch.argmax(outputs.start_logits)
        end_idx = torch.argmax(outputs.end_logits) + 1

    answer_ids = inputs["input_ids"][0][start_idx:end_idx]
    answer = tokenizer.decode(answer_ids, skip_special_tokens=True)
    return answer

# Run QA on the sample note
qa_results = {'ServiceDate': "03/12/2025", 'DaywiseNotes': sample_note}
for i, question in enumerate(questions):
    qa_results[f"Q{i+1}"] = answer_question(sample_note, question)

# Create DataFrame
df_9 = pd.DataFrame([qa_results])
df_9

,ServiceDate,DaywiseNotes,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10
0,03/12/2025,\nDate: 3/12/2025\n08:00 – Opening shift note....,when were the restraints initiated? date : 3 /...,side rails,"no signs of distress, sob, or pain",patient instructed on plan of care and to call...,,patient is awake and alert,"distress, sob, or pain",no mention of renewal,no mention of discontinued. when were the rest...,patient is awake and alert


### **Sample Notes - 10 : Day-10**

In [123]:
sample_note = """
Date: 3/13/2025
09:58 – Resident rounds by MD Cruz. Patient mittens removed, patient pleasant. No new orders.
10:15 – In process of obtaining SCDs.
13:50 – Attempted to give 1400 medications. Family at bedside requested delay as patient had just fallen asleep. RN acknowledged.
14:34 – Follow-up visit; patient still asleep. New family member requested RN to return later.
16:10 – IV removal: 22G in right forearm and 20G in left forearm removed with sterile technique. Catheters intact, pressure dressing applied. Patient tolerated procedure well.
16:14 – Spoke with Dr. Siddiqui via phone; approved change in discharge disposition to “Home.”
16:28 – Transportation arrived at bedside.
16:41 – Discharge completed. Instructions given and all questions addressed. Medication reconciliation form provided. Patient escorted to vehicle via gurney with all belongings, no signs of pain or distress. Accompanied by GreenMed transport.
"""

# QA function (Extractive)
def answer_question(note, question):
    inputs = tokenizer.encode_plus(question, note, return_tensors="pt", truncation=True)

    with torch.no_grad():
        outputs = model(**inputs)
        start_idx = torch.argmax(outputs.start_logits)
        end_idx = torch.argmax(outputs.end_logits) + 1

    answer_ids = inputs["input_ids"][0][start_idx:end_idx]
    answer = tokenizer.decode(answer_ids, skip_special_tokens=True)
    return answer

# Run QA on the sample note
qa_results = {'ServiceDate': "03/13/2025", 'DaywiseNotes': sample_note}
for i, question in enumerate(questions):
    qa_results[f"Q{i+1}"] = answer_question(sample_note, question)

# Create DataFrame
df_10 = pd.DataFrame([qa_results])
df_10

,ServiceDate,DaywiseNotes,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10
0,03/13/2025,\nDate: 3/13/2025\n09:58 – Resident rounds by ...,answer only if ' initiated ' is mentioned. oth...,"catheters intact, pressure dressing",,,,,,no mention of renewal,no mention of discontinued. when were the rest...,"patient mittens removed, patient pleasant. no ..."


## **-----------------DistilBERT Final Q&A Results---------------------**

In [124]:
# Step 1: Create a list of all dataframes
all_dfs = [df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8, df_9, df_10]

# Step 2: Concatenate into a single DataFrame
df_combined_bert = pd.concat(all_dfs, ignore_index=True)

# Step 3: Display the combined result
display(df_combined_bert)

,ServiceDate,DaywiseNotes,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10
0,03/03/2025,\nDate: 3/3/2025\n14:38 - Patient's BP elevate...,,soft restraints,,,np okpan,,,no mention of renewal,answer only if ' discontinue ' or ' discontinu...,
1,03/04/2025,\nDate: 3/4/2025\n03:40 - Patient is sleeping....,,,,,,,,,answer only if ' discontinue ' or ' discontinu...,
2,03/05/2025,\nDate: 3/5/2025\n07:30 - Patient sleeping wit...,3 / 5 / 2025,medications given as ordered,medications given as ordered,,sitter,no signs of distress,medications given as ordered. linen changed. p...,no mention of renewal,no mention of discontinued. when were the rest...,confused
3,03/06/2025,\nDate: 3/6/2025\n07:30 - Opening shift note. ...,when were the restraints initiated? date : 3 /...,side rails up x2,no signs of distress or shortness of breath. p...,charge nurse david notified,charge nurse david notified,patient awake and alert,no signs of distress or shortness of breath,no mention of renewal,no mention of discontinued. when were the rest...,patient awake and alert
4,03/07/2025,\nDate: 3/7/2025\n07:09 - Closing shift note. ...,when were the restraints initiated? date : 3 /...,safety precautions,all safety precautions in place,all safety precautions,night shift nurse elias,all safety precautions in place,safety precautions,no mention of renewal,no mention of discontinued,"awake, confused, and uncooperative"
5,03/08/2025,\nDate: 3/8/2025\n06:26 - Blood glucose 53 (fi...,when were the restraints initiated? date : 3 /...,sitter,"patient sleeping but easily aroused, not coope...",close monitoring,md paged,"patient sleeping but easily aroused, not coope...",sleeping,no mention of renewal,no mention of discontinued,sleeping but easily aroused
6,03/10/2025,\nDate: 3/10/2025\n13:58 – Resident MD Moreno ...,when were the restraints initiated? date : 3 /...,medications,patient was asleep most of the day and became ...,informed that morning medications were held as...,tele psych consult,tele psych consult,medications were held as patient was asleep mo...,no mention of renewal,no mention of discontinued,
7,03/11/2025,\nDate: 3/11/2025\n10:36 – Resident MD Cruz Mo...,when were the restraints initiated? date : 3 /...,morning medications,patient agitation and combativeness,"patient is more awake today, took morning medi...",dr. zhang,"patient is more awake today, took morning medi...","morning medications, noncombative, and less ye...",no mention of renewal,no mention of discontinued,yelling
8,03/12/2025,\nDate: 3/12/2025\n08:00 – Opening shift note....,when were the restraints initiated? date : 3 /...,side rails,"no signs of distress, sob, or pain",patient instructed on plan of care and to call...,,patient is awake and alert,"distress, sob, or pain",no mention of renewal,no mention of discontinued. when were the rest...,patient is awake and alert
9,03/13/2025,\nDate: 3/13/2025\n09:58 – Resident rounds by ...,answer only if ' initiated ' is mentioned. oth...,"catheters intact, pressure dressing",,,,,,no mention of renewal,no mention of discontinued. when were the rest...,"patient mittens removed, patient pleasant. no ..."


In [125]:
from datetime import datetime

# Generate today's date string
today_str = datetime.today().strftime('%m%d%Y')

# Define filename with today's date
filename = f'DistilBERT_QA-Answers-{today_str}.csv'

# Save DataFrame to CSV
df_combined_bert.to_csv(filename, index=False)

# If using Google Colab, download the file
from google.colab import files
files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## **BART - Question & Answer Model**

In [126]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch
import pandas as pd

In [127]:
# Load the FLAN-T5 model (or any T5 QA model)
model_name = "valhalla/bart-large-finetuned-squadv1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels will be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels will be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels will be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels will be overwritten to 2.


In [128]:
# Finalized restraint-related questions
questions = [
    "Answer only if 'initiated' is mentioned. Otherwise, reply 'No mention of initiated. When were the restraints initiated?",
    "What type of restraints were applied?",
    "Why were the restraints applied?",
    "Was a physician or provider notified or present at the time of restraint?",
    "Who was the provider or nurse involved in the restraint assessment or order?",
    "Was the bed alarm used for patient safety?",
    "What comfort measures were attempted before or during restraint?",
    "Answer only if 'renew' or 'renewed' or 'renewal' is mentioned. Otherwise, reply 'No mention of renewal. When was the restraint order renewed (if applicable)?",
    "Answer only if 'discontinue' or 'discontinued' is mentioned. Otherwise, reply 'No mention of discontinued. When were the restraints discontinued?",
    "What was the patient’s behavior at the time of restraint discontinuation?"
]

### **Sample Notes - 1 : Day-1**

In [129]:
sample_note = """
Date: 3/3/2025
14:38 - Patient's BP elevated. RN notified MD Tummala; orders initiated.
14:53 - Labetalol held. BP 139/99 mmHg.
15:38 - Patient keeps getting out of bed. Restless and agitated. RN notified MD Tummala again; medication ordered for comfort.
16:10 - Patient still trying to get out of bed. Ativan administered; no effect. Plan to initiate restraints if non-cooperative.
17:05 - NP Okpan assessed the patient.
17:20 - Soft wrist and ankle restraints initiated due to combative behavior and non-compliance. Patient pulling out lines, repeatedly getting out of bed. Neuro: A&OX4. PERRLA normal. CNL noted.
17:47 - Restraints reiterated. Patient now A&OX2 (oriented to self and place).
17:48 - MD Tummala at bedside. Patient yelling. Additional orders initiated.
18:33 - Patient remains restless in soft restraints. Pulses palpable. VSS. Patient confused.
19:30 - Patient admitted from ED with fall injury (facial contusion, edema). No respiratory distress. PMH: TBI, HTN, DM, schizophrenia, seizures. IV placed. Monitoring initiated.
19:30 - Patient continuesa to be restless in soft restraints.
21:40 - Patient calm and sleepy. Initially placed in 4-point restraints due to ER behavior (agitation, combative, pulling lines, non-compliance). 4-point restraints discontinued. Soft restraints applied to both wrists. Medical non-violent restraint order obtained from NP Okpan. Continued restraint monitoring planned. Restraint assessment completed. Family not reachable.
"""

# QA function
def answer_question(note, question):
    inputs = tokenizer.encode_plus(question, note, return_tensors="pt", truncation=True, max_length=512)

    with torch.no_grad():
        outputs = model(**inputs)
        start_idx = torch.argmax(outputs.start_logits)
        end_idx = torch.argmax(outputs.end_logits)

        # Ensure valid range
        if start_idx > end_idx:
            return "No clear answer found."

    answer_ids = inputs["input_ids"][0][start_idx:end_idx + 1]
    answer = tokenizer.decode(answer_ids, skip_special_tokens=True).strip()
    return answer

# Run QA
qa_results = {'ServiceDate': "03/03/2025", 'DaywiseNotes': sample_note}
for i, question in enumerate(questions):
    qa_results[f"Q{i+1}"] = answer_question(sample_note, question)

# Create DataFrame
df_1 = pd.DataFrame([qa_results])
df_1

,ServiceDate,DaywiseNotes,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10
0,03/03/2025,\nDate: 3/3/2025\n14:38 - Patient's BP elevate...,3/3/2025,Soft wrist and ankle,combative behavior and non-compliance,No clear answer found.,NP Okpan,No clear answer found.,medication,No mention of renewal. When was the restraint ...,3/3/2025\n14:38 - Patient's BP elevated. RN no...,No clear answer found.


### **Sample Notes - 2 : Day-2**

In [130]:
sample_note = """
Date: 3/4/2025
03:40 - Patient is sleeping. No signs of distress or SOB. Attempted to wake but very sleepy. Sitter bedside. Bed alarm on for safety. Monitoring continues.
07:40 - Soft restraints discontinued. Patient no longer pulling lines/tubes. Still confused but reorientable. Skin intact. Patient A&Ox2. Offered nutrition, hydration, repositioning, hygiene. Safety precautions in place: bed in lowest position, 2 side rails, call light reachable, bed alarm on, sitter present.
10:20 - Patient increasingly agitated, pulling lines, trying to get out of bed, hitting and kicking staff. Reorientation and distractions failed. Ativan administered. MD notified. Sitter at bedside.
10:30 - Agitation continues. Patient yelling and attempting to strike staff. Dr. Testa notified and new orders received.
11:00 - Dr. Testa at bedside.
11:10 - Patient placed in 4-point soft restraints due to extreme agitation and combative behavior. All comfort measures failed. Dr. Testa notified. Order for restraints obtained.
        Patient educated on use and safety. Restraints applied.
11:48 - Dr. Testa on unit. New order placed.
13:10 - Restraints continued. Agitation persists. All comfort measures failed. Skin intact. Patient A&Ox1, unable to comprehend instructions due to ALOC/agitation. Safety precautions maintained. Nutrition, repositioning, and hygiene offered. Sitter present.
15:10 - Restraint renewal order placed. Patient continues pulling at lines and exhibiting aggressive behavior. All comfort measures ineffective. A&Ox1. Patient unable to comprehend instructions. Skin intact. All safety protocols followed.
17:10 - Restraints continued. No improvement in agitation. Measures repeated. Skin intact. A&Ox1. Repositioned and hydrated. Sitter present.
19:10 - Another restraint renewal order. Same behavioral pattern. Skin intact. A&Ox1. Patient unable to understand safety instructions. Continued monitoring and care.
21:10 - Restraints continued. Behavior unchanged. No redness or skin breakdown. Patient remains A&Ox. Safety precautions enforced.
23:00 - Soft restraints discontinued. Patient AOx2, still mildly confused but no longer combative or pulling IVs. Skin assessed — no issues. Patient reoriented and repositioned. Hydration/nutrition offered. Patient resting. Bed in lowest position, brakes on, side rails up, call light reachable, sitter at bedside.
"""

# QA function
def answer_question(note, question):
    inputs = tokenizer.encode_plus(question, note, return_tensors="pt", truncation=True, max_length=512)

    with torch.no_grad():
        outputs = model(**inputs)
        start_idx = torch.argmax(outputs.start_logits)
        end_idx = torch.argmax(outputs.end_logits)

        # Ensure valid range
        if start_idx > end_idx:
            return "No clear answer found."

    answer_ids = inputs["input_ids"][0][start_idx:end_idx + 1]
    answer = tokenizer.decode(answer_ids, skip_special_tokens=True).strip()
    return answer

# Run QA
qa_results = {'ServiceDate': "03/03/2025", 'DaywiseNotes': sample_note}
for i, question in enumerate(questions):
    qa_results[f"Q{i+1}"] = answer_question(sample_note, question)

# Create DataFrame
df_2 = pd.DataFrame([qa_results])
df_2

,ServiceDate,DaywiseNotes,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10
0,03/03/2025,\nDate: 3/4/2025\n03:40 - Patient is sleeping....,3/4/2025,4-point soft restraints,extreme agitation and combative behavior,No clear answer found.,No clear answer found.,Bed alarm on for safety,comfort measures failed,No clear answer found.,3/4/2025,hitting and kicking staff


### **Sample Notes - 3 : Day-3**

In [131]:
sample_note = """
Date: 3/5/2025
07:30 - Patient sleeping with eyes closed. No signs of distress. Sitter at bedside.
09:44 - Breakfast fed and tolerated well. Medications given as ordered. Linen changed. Patient remains confused, unable to state name, date of birth, or location. Continues attempting to get out of bed. Sitter at bedside.
10:26 - Patient yelling, confused, and trying to get out of bed. Sitter at bedside.
18:25 - Patient agitated, yelling, and attempting to get out of bed. Medicated with Ativan. Plan of care continued.
20:34 - Patient agitated and trying to get out of bed. Hydration and repositioning offered. Patient was redirected and educated on use of telemetry box and gown. Patient cooperated and rested after intervention. Sitter at bedside. Plan of care continued.
"""

# QA function
def answer_question(note, question):
    inputs = tokenizer.encode_plus(question, note, return_tensors="pt", truncation=True, max_length=512)

    with torch.no_grad():
        outputs = model(**inputs)
        start_idx = torch.argmax(outputs.start_logits)
        end_idx = torch.argmax(outputs.end_logits)

        # Ensure valid range
        if start_idx > end_idx:
            return "No clear answer found."

    answer_ids = inputs["input_ids"][0][start_idx:end_idx + 1]
    answer = tokenizer.decode(answer_ids, skip_special_tokens=True).strip()
    return answer

# Run QA
qa_results = {'ServiceDate': "03/03/2025", 'DaywiseNotes': sample_note}
for i, question in enumerate(questions):
    qa_results[f"Q{i+1}"] = answer_question(sample_note, question)

# Create DataFrame
df_3 = pd.DataFrame([qa_results])
df_3

,ServiceDate,DaywiseNotes,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10
0,03/03/2025,\nDate: 3/5/2025\n07:30 - Patient sleeping wit...,3/5/2025,telemetry box and gown,"unable to state name, date of birth, or location",No clear answer found.,No clear answer found.,telemetry box,Hydration and repositioning,No clear answer found.,3/5/2025,agitated


### **Sample Notes - 4 : Day-4**

In [132]:
sample_note = """
Date: 3/6/2025
07:30 - Opening shift note. Assumed care of patient. Patient awake and alert. No signs of distress or shortness of breath. Patient out of bed, yelling at CNA, attempting to leave room. Charge nurse David notified. Bed in lowest locked position, side rails up x2, call light within reach. Care ongoing.
08:05 - Report given to Edididong RN. Patient at bedside eating breakfast with no signs of distress or pain. Care ongoing.
19:58 - Patient agitated, pulling on leads, trying to jump off bed, hitting aide.
"""

# QA function
def answer_question(note, question):
    inputs = tokenizer.encode_plus(question, note, return_tensors="pt", truncation=True, max_length=512)

    with torch.no_grad():
        outputs = model(**inputs)
        start_idx = torch.argmax(outputs.start_logits)
        end_idx = torch.argmax(outputs.end_logits)

        # Ensure valid range
        if start_idx > end_idx:
            return "No clear answer found."

    answer_ids = inputs["input_ids"][0][start_idx:end_idx + 1]
    answer = tokenizer.decode(answer_ids, skip_special_tokens=True).strip()
    return answer

# Run QA
qa_results = {'ServiceDate': "03/03/2025", 'DaywiseNotes': sample_note}
for i, question in enumerate(questions):
    qa_results[f"Q{i+1}"] = answer_question(sample_note, question)

# Create DataFrame
df_4 = pd.DataFrame([qa_results])
df_4

,ServiceDate,DaywiseNotes,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10
0,03/03/2025,\nDate: 3/6/2025\n07:30 - Opening shift note. ...,3/6/2025,side rails up x2,"agitated, pulling on leads, trying to jump off...",Charge nurse David notified,Edididong RN,call light,call light within reach,No clear answer found.,3/6,"agitated, pulling on leads, trying to jump off..."


### **Sample Notes - 5 : Day-5**

In [133]:
sample_note = """
Date: 3/7/2025
07:09 - Closing shift note. Care endorsed to day shift.
19:20 - Opening shift note. Assumed care of patient. Patient is awake, confused, and uncooperative. No signs of distress, shortness of breath, or pain. All safety precautions in place. Call light within reach. Sitter at bedside.
20:48 - Handoff report given to night shift nurse Elias for continued care.
"""

# QA function
def answer_question(note, question):
    inputs = tokenizer.encode_plus(question, note, return_tensors="pt", truncation=True, max_length=512)

    with torch.no_grad():
        outputs = model(**inputs)
        start_idx = torch.argmax(outputs.start_logits)
        end_idx = torch.argmax(outputs.end_logits)

        # Ensure valid range
        if start_idx > end_idx:
            return "No clear answer found."

    answer_ids = inputs["input_ids"][0][start_idx:end_idx + 1]
    answer = tokenizer.decode(answer_ids, skip_special_tokens=True).strip()
    return answer

# Run QA
qa_results = {'ServiceDate': "03/03/2025", 'DaywiseNotes': sample_note}
for i, question in enumerate(questions):
    qa_results[f"Q{i+1}"] = answer_question(sample_note, question)

# Create DataFrame
df_5 = pd.DataFrame([qa_results])
df_5

,ServiceDate,DaywiseNotes,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10
0,03/03/2025,\nDate: 3/7/2025\n07:09 - Closing shift note. ...,3/7/2025,All safety precautions,uncooperative,,Elias,All safety precautions,All safety precautions,No clear answer found.,3/7/2025,uncooperative


### **Sample Notes - 6 : Day-6**

In [134]:
sample_note = """
Date: 3/8/2025
06:26 - Blood glucose 53 (first drop into the 40s). Patient sleeping but easily aroused, not cooperating with drinking juice. D50 IV administered as ordered. MD paged. Patient under close monitoring.
18:10 - Report called to Joyce, RN. Patient to be transferred to Room 220A.
18:19 - Patient transferred to unit in stable condition. Sitter at bedside.
"""

# QA function
def answer_question(note, question):
    inputs = tokenizer.encode_plus(question, note, return_tensors="pt", truncation=True, max_length=512)

    with torch.no_grad():
        outputs = model(**inputs)
        start_idx = torch.argmax(outputs.start_logits)
        end_idx = torch.argmax(outputs.end_logits)

        # Ensure valid range
        if start_idx > end_idx:
            return "No clear answer found."

    answer_ids = inputs["input_ids"][0][start_idx:end_idx + 1]
    answer = tokenizer.decode(answer_ids, skip_special_tokens=True).strip()
    return answer

# Run QA
qa_results = {'ServiceDate': "03/03/2025", 'DaywiseNotes': sample_note}
for i, question in enumerate(questions):
    qa_results[f"Q{i+1}"] = answer_question(sample_note, question)

# Create DataFrame
df_6 = pd.DataFrame([qa_results])
df_6

,ServiceDate,DaywiseNotes,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10
0,03/03/2025,\nDate: 3/8/2025\n06:26 - Blood glucose 53 (fi...,3/8/2025,No clear answer found.,"sleeping but easily aroused, not cooperating w...",MD,"Joyce, RN",,IV,No clear answer found.,3/8,sleeping


### **Sample Notes - 7 : Day-7**

In [135]:
sample_note = """
Date: 3/10/2025
13:58 – Resident MD Moreno rounding. Informed that morning medications were held as patient was asleep most of the day and became verbally incomprehensible when awakened. Patient yelled "I sleep" and resisted touch. Order placed for tele psych consult.
14:07 – Tele psych completed. MD Oliver recommended increasing Seroquel dosage to 300 mg and advised communication with resident MD. MD Cruz Moreno was informed.
"""

# QA function
def answer_question(note, question):
    inputs = tokenizer.encode_plus(question, note, return_tensors="pt", truncation=True, max_length=512)

    with torch.no_grad():
        outputs = model(**inputs)
        start_idx = torch.argmax(outputs.start_logits)
        end_idx = torch.argmax(outputs.end_logits)

        # Ensure valid range
        if start_idx > end_idx:
            return "No clear answer found."

    answer_ids = inputs["input_ids"][0][start_idx:end_idx + 1]
    answer = tokenizer.decode(answer_ids, skip_special_tokens=True).strip()
    return answer

# Run QA
qa_results = {'ServiceDate': "03/03/2025", 'DaywiseNotes': sample_note}
for i, question in enumerate(questions):
    qa_results[f"Q{i+1}"] = answer_question(sample_note, question)

# Create DataFrame
df_7 = pd.DataFrame([qa_results])
df_7

,ServiceDate,DaywiseNotes,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10
0,03/03/2025,\nDate: 3/10/2025\n13:58 – Resident MD Moreno ...,3/10/2025,touch,patient was asleep most of the day and became ...,MD Cruz Moreno,Resident MD Moreno,patient was asleep,touch,3/10,3/10,asleep most of the day and became verbally inc...


### **Sample Notes - 8 : Day-8**

In [136]:
sample_note = """
Date: 3/11/2025
10:36 – Resident MD Cruz Moreno rounding. Patient is more awake today, took morning medications, noncombative, and less yelling compared to previous days. New orders received.
21:30 – New orders received from Dr. Zhang due to patient agitation and combativeness.
"""

# QA function
def answer_question(note, question):
    inputs = tokenizer.encode_plus(question, note, return_tensors="pt", truncation=True, max_length=512)

    with torch.no_grad():
        outputs = model(**inputs)
        start_idx = torch.argmax(outputs.start_logits)
        end_idx = torch.argmax(outputs.end_logits)

        # Ensure valid range
        if start_idx > end_idx:
            return "No clear answer found."

    answer_ids = inputs["input_ids"][0][start_idx:end_idx + 1]
    answer = tokenizer.decode(answer_ids, skip_special_tokens=True).strip()
    return answer

# Run QA
qa_results = {'ServiceDate': "03/03/2025", 'DaywiseNotes': sample_note}
for i, question in enumerate(questions):
    qa_results[f"Q{i+1}"] = answer_question(sample_note, question)

# Create DataFrame
df_8 = pd.DataFrame([qa_results])
df_8

,ServiceDate,DaywiseNotes,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10
0,03/03/2025,\nDate: 3/11/2025\n10:36 – Resident MD Cruz Mo...,3/11/2025,patient agitation,patient agitation and combativeness,Resident MD Cruz Moreno rounding,Resident MD Cruz Moreno,,medications,3/11,3/11/2025,"noncombative, and less yelling"


### **Sample Notes - 9 : Day-9**

In [137]:
sample_note = """
Date: 3/12/2025
08:00 – Opening shift note. Assumed care of patient. Patient is awake and alert. Bed is in lowest locked position with side rails up x2 and call light within reach. No signs of distress, SOB, or pain. Patient instructed on plan of care and to call for assistance as needed. Sitter is at bedside.
10:00 – Per Dr. Cruz, Lisinopril was held.
17:31 – Tele box returned to monitor tech.
"""

# QA function
def answer_question(note, question):
    inputs = tokenizer.encode_plus(question, note, return_tensors="pt", truncation=True, max_length=512)

    with torch.no_grad():
        outputs = model(**inputs)
        start_idx = torch.argmax(outputs.start_logits)
        end_idx = torch.argmax(outputs.end_logits)

        # Ensure valid range
        if start_idx > end_idx:
            return "No clear answer found."

    answer_ids = inputs["input_ids"][0][start_idx:end_idx + 1]
    answer = tokenizer.decode(answer_ids, skip_special_tokens=True).strip()
    return answer

# Run QA
qa_results = {'ServiceDate': "03/03/2025", 'DaywiseNotes': sample_note}
for i, question in enumerate(questions):
    qa_results[f"Q{i+1}"] = answer_question(sample_note, question)

# Create DataFrame
df_9 = pd.DataFrame([qa_results])
df_9

,ServiceDate,DaywiseNotes,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10
0,03/03/2025,\nDate: 3/12/2025\n08:00 – Opening shift note....,3/12/2025,No clear answer found.,No clear answer found.,No clear answer found.,Dr. Cruz,call light,Lisinopril was held,No clear answer found.,3/12/2025,awake and alert


### **Sample Notes - 10 : Day-10**

In [138]:
sample_note = """
Date: 3/13/2025
09:58 – Resident rounds by MD Cruz. Patient mittens removed, patient pleasant. No new orders.
10:15 – In process of obtaining SCDs.
13:50 – Attempted to give 1400 medications. Family at bedside requested delay as patient had just fallen asleep. RN acknowledged.
14:34 – Follow-up visit; patient still asleep. New family member requested RN to return later.
16:10 – IV removal: 22G in right forearm and 20G in left forearm removed with sterile technique. Catheters intact, pressure dressing applied. Patient tolerated procedure well.
16:14 – Spoke with Dr. Siddiqui via phone; approved change in discharge disposition to “Home.”
16:28 – Transportation arrived at bedside.
16:41 – Discharge completed. Instructions given and all questions addressed. Medication reconciliation form provided. Patient escorted to vehicle via gurney with all belongings, no signs of pain or distress. Accompanied by GreenMed transport.
"""

# QA function
def answer_question(note, question):
    inputs = tokenizer.encode_plus(question, note, return_tensors="pt", truncation=True, max_length=512)

    with torch.no_grad():
        outputs = model(**inputs)
        start_idx = torch.argmax(outputs.start_logits)
        end_idx = torch.argmax(outputs.end_logits)

        # Ensure valid range
        if start_idx > end_idx:
            return "No clear answer found."

    answer_ids = inputs["input_ids"][0][start_idx:end_idx + 1]
    answer = tokenizer.decode(answer_ids, skip_special_tokens=True).strip()
    return answer

# Run QA
qa_results = {'ServiceDate': "03/03/2025", 'DaywiseNotes': sample_note}
for i, question in enumerate(questions):
    qa_results[f"Q{i+1}"] = answer_question(sample_note, question)

# Create DataFrame
df_10 = pd.DataFrame([qa_results])
df_10

,ServiceDate,DaywiseNotes,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10
0,03/03/2025,\nDate: 3/13/2025\n09:58 – Resident rounds by ...,3/13/2025,pressure dressing applied,Why were the restraints applied?\nDate: 3/13/2...,,Who was the provider or nurse involved in the ...,Family at bedside requested delay as patient h...,"mittens removed, patient pleasant. No new orde...",No clear answer found.,No clear answer found.,pleasant


## **-------------------BART Final Q&A Results-----------------------**

In [139]:
# Step 1: Create a list of all dataframes
all_dfs = [df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8, df_9, df_10]

# Step 2: Concatenate into a single DataFrame
df_combined_bart = pd.concat(all_dfs, ignore_index=True)

# Step 3: Display the combined result
display(df_combined_bart)

,ServiceDate,DaywiseNotes,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10
0,03/03/2025,\nDate: 3/3/2025\n14:38 - Patient's BP elevate...,3/3/2025,Soft wrist and ankle,combative behavior and non-compliance,No clear answer found.,NP Okpan,No clear answer found.,medication,No mention of renewal. When was the restraint ...,3/3/2025\n14:38 - Patient's BP elevated. RN no...,No clear answer found.
1,03/03/2025,\nDate: 3/4/2025\n03:40 - Patient is sleeping....,3/4/2025,4-point soft restraints,extreme agitation and combative behavior,No clear answer found.,No clear answer found.,Bed alarm on for safety,comfort measures failed,No clear answer found.,3/4/2025,hitting and kicking staff
2,03/03/2025,\nDate: 3/5/2025\n07:30 - Patient sleeping wit...,3/5/2025,telemetry box and gown,"unable to state name, date of birth, or location",No clear answer found.,No clear answer found.,telemetry box,Hydration and repositioning,No clear answer found.,3/5/2025,agitated
3,03/03/2025,\nDate: 3/6/2025\n07:30 - Opening shift note. ...,3/6/2025,side rails up x2,"agitated, pulling on leads, trying to jump off...",Charge nurse David notified,Edididong RN,call light,call light within reach,No clear answer found.,3/6,"agitated, pulling on leads, trying to jump off..."
4,03/03/2025,\nDate: 3/7/2025\n07:09 - Closing shift note. ...,3/7/2025,All safety precautions,uncooperative,,Elias,All safety precautions,All safety precautions,No clear answer found.,3/7/2025,uncooperative
5,03/03/2025,\nDate: 3/8/2025\n06:26 - Blood glucose 53 (fi...,3/8/2025,No clear answer found.,"sleeping but easily aroused, not cooperating w...",MD,"Joyce, RN",,IV,No clear answer found.,3/8,sleeping
6,03/03/2025,\nDate: 3/10/2025\n13:58 – Resident MD Moreno ...,3/10/2025,touch,patient was asleep most of the day and became ...,MD Cruz Moreno,Resident MD Moreno,patient was asleep,touch,3/10,3/10,asleep most of the day and became verbally inc...
7,03/03/2025,\nDate: 3/11/2025\n10:36 – Resident MD Cruz Mo...,3/11/2025,patient agitation,patient agitation and combativeness,Resident MD Cruz Moreno rounding,Resident MD Cruz Moreno,,medications,3/11,3/11/2025,"noncombative, and less yelling"
8,03/03/2025,\nDate: 3/12/2025\n08:00 – Opening shift note....,3/12/2025,No clear answer found.,No clear answer found.,No clear answer found.,Dr. Cruz,call light,Lisinopril was held,No clear answer found.,3/12/2025,awake and alert
9,03/03/2025,\nDate: 3/13/2025\n09:58 – Resident rounds by ...,3/13/2025,pressure dressing applied,Why were the restraints applied?\nDate: 3/13/2...,,Who was the provider or nurse involved in the ...,Family at bedside requested delay as patient h...,"mittens removed, patient pleasant. No new orde...",No clear answer found.,No clear answer found.,pleasant


In [140]:
from datetime import datetime

# Generate today's date string
today_str = datetime.today().strftime('%m%d%Y')

# Define filename with today's date
filename = f'BART_QA-Answers-{today_str}.csv'

# Save DataFrame to CSV
df_combined_bart.to_csv(filename, index=False)

# If using Google Colab, download the file
from google.colab import files
files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>